# Plan

Create an agent that consistently beats our previously submitted decision_tree agent and all the given environment agents. The agent should not be a specifically crafted based on the decision tree to beat, but rather generic.

## Load all the agents I want to benchmark against

In [1]:
from kaggle_environments import make
from kaggle_environments.envs.rps.agents import agents

print(agents)

Loading environment football failed: No module named 'gfootball'
{'rock': <function rock at 0x7f3a40de9790>, 'paper': <function paper at 0x7f3a40ae1700>, 'scissors': <function scissors at 0x7f3a40ae1820>, 'copy_opponent': <function copy_opponent at 0x7f3a40ae18b0>, 'reactionary': <function reactionary at 0x7f3a40ae1940>, 'counter_reactionary': <function counter_reactionary at 0x7f3a40ae19d0>, 'statistical': <function statistical at 0x7f3a40ae1a60>}


In [2]:
# adding random agent
import random

def random_agent(observation, configuration):
    return random.randint(0, configuration.signs-1)

In [3]:
import sys
import os

os.getcwd()

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/wolle/git/kaggle/rock-paper-scissors/rps_basic_decision_tree')

from submission import tree_agent

In [4]:
import inspect
lines = inspect.getsource(tree_agent)
print(lines)


def tree_agent(observation, configuration):
    global opponent_observations
    global my_actions
    global sample_size
    
    if observation.step == 0:
        action = 1
        my_actions.append(action)
        return action
    
    elif observation.step <= 2 * sample_size:
        opponent_observations.append(observation.lastOpponentAction)
        action = random.randint(0, configuration.signs-1)
        my_actions.append(action)
        return action
        
    else:
        opponent_observations.append(observation.lastOpponentAction)
        
        action = action_decision_tree_dev(
            opponent_observations,
            my_actions,
            sample_size,
            configuration
        )
        
        my_actions.append(action)
        return action



## Step 1: A decision tree counter

In [5]:
%%writefile beat_tree.py

"""
This agent uses a decision tree to predict it's own action, in order to guess what the opponent would predict,
and then returns the counter of the counter in order beat the oponents best action. Pretty meta...
"""

import random
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier

sample_size = 5 # 5 for mirroring te opponent tree
tree_depth = 4 # 4 for mirroring the opponent tree
opponent_observations = []
my_actions = []

def make_train_set_v2(opponent_observations, my_actions, sample_size):
    observed = np.array([list(x) for x in zip(my_actions, opponent_observations)])
    prediction_sample = observed[-sample_size:].reshape(1, sample_size * 2)

    splits = TimeSeriesSplit(
        max_train_size=sample_size,
        n_splits=len(observed) - sample_size
    )
        
    X = np.array([
            observed[train_index] for train_index, _ in splits.split(observed)
    ])
    
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

    Y = np.array([
            observed[y_index][0][0] for _, y_index in splits.split(observed)
    ])

    Y = Y.reshape(Y.shape[0], 1)
    
    return X, Y, prediction_sample

def action_beat_tree(opponent_observations, my_actions,
    sample_size, configuration=None):

    X, Y, prediction_sample = make_train_set_v2(opponent_observations, my_actions, sample_size)

    model = DecisionTreeClassifier(
        criterion="entropy",
        max_depth = tree_depth,
    )

    model.fit(X, Y)

    prediction = model.predict(prediction_sample)
    
    # action is what beats the opponents prediciton
    action = (prediction + 2) % (configuration.signs if configuration else 3)
    action = int(action[0])
        
    return action

def beat_tree_agent(observation, configuration):
    global opponent_observations
    global my_actions
    global sample_size
    
    if observation.step == 0:
        action = 1
        my_actions.append(action)
        return action
    
    elif observation.step <= 2 * sample_size:
        opponent_observations.append(observation.lastOpponentAction)
        action = random.randint(0, configuration.signs-1)
        my_actions.append(action)
        return action
        
    else:
        opponent_observations.append(observation.lastOpponentAction)
        
        action = action_beat_tree(
            opponent_observations,
            my_actions,
            sample_size,
            configuration
        )
        
        my_actions.append(action)
        return action

Overwriting beat_tree.py


In [6]:
%run -i 'beat_tree.py'

In [7]:
# a simple test

a = action_beat_tree(
    opponent_observations = [0, 1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 2], #opp mirros me
    my_actions = [1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 2, 0],
    sample_size = 5,
    configuration=None
)  #expected outcome: 0

print(a, sep="\n")

0


In [8]:
# run against agents
from kaggle_environments import evaluate

env = make("rps", configuration={"episodeSteps": 100}, debug=True)

env.run([beat_tree_agent, tree_agent])
env.render(mode="ipython", width=600, height=600)

In [9]:
evaluate(
    environment="rps",
    configuration={"episodeSteps": 100},
    agents=[beat_tree_agent, tree_agent],
    num_episodes=1
)

[[0, 0]]

In [10]:
"""Ok, the agent consistently beats my tree by design, but it sucks against statistical agent,
rock agent, and friends... So we need to be creative, to utilize it."""

'Ok, the agent consistently beats my tree by design, but it sucks against statistical agent,\nrock agent, and friends... So we need to be creative, to utilize it.'

In [11]:
"""Actually, this one is not even consistent in beating it's tree counterpart as observed after running 
several episodes. Well, let's forget avobut this approach for now."""

"Actually, this one is not even consistent in beating it's tree counterpart as observed after running \nseveral episodes. Well, let's forget avobut this approach for now."